# 交易数据

交易类数据提供股票的交易行情数据，通过简单的接口调用可获取相应的DataFrame格式数据，主要包括以下类别：

    历史行情数据
    复权历史数据
    实时行情数据
    历史分笔数据
    实时报价数据
    当日历史分笔
    大盘指数列表
    大单交易数据

历史行情
在新版0.5.6中，已经新增了一个接口：get_k_data，具体使用方法请参考tushare公众号文章《全新的免费行情数据接口》 ，建议使用全新接口。

获取个股历史交易数据（包括均线数据），可以通过参数设置获取日k线、周k线、月k线，以及5分钟、15分钟、30分钟和60分钟k线数据。本接口只能获取近3年的日线数据，适合搭配均线数据进行选股和分析，如果需要全部历史数据，请调用下一个接口get_h_data()。

参数说明：

    code：股票代码，即6位数字代码，或者指数代码（sh=上证指数 sz=深圳成指 hs300=沪深300指数 sz50=上证50 zxb=中小板 cyb=创业板）
    start：开始日期，格式YYYY-MM-DD
    end：结束日期，格式YYYY-MM-DD
    ktype：数据类型，D=日k线 W=周 M=月 5=5分钟 15=15分钟 30=30分钟 60=60分钟，默认为D
    retry_count：当网络异常后重试次数，默认为3
    pause:重试时停顿秒数，默认为0

返回值说明：

    date：日期
    open：开盘价
    high：最高价
    close：收盘价
    low：最低价
    volume：成交量
    price_change：价格变动
    p_change：涨跌幅
    ma5：5日均价
    ma10：10日均价
    ma20:20日均价
    v_ma5:5日均量
    v_ma10:10日均量
    v_ma20:20日均量
    turnover:换手率[注：指数无此项]

调用方法：
```
import tushare as ts
ts.get_hist_data('600848') #一次性获取全部日k线数据
```

其他：
```
ts.get_hist_data('600848', ktype='W') #获取周k线数据
ts.get_hist_data('600848', ktype='M') #获取月k线数据
ts.get_hist_data('600848', ktype='5') #获取5分钟k线数据
ts.get_hist_data('600848', ktype='15') #获取15分钟k线数据
ts.get_hist_data('600848', ktype='30') #获取30分钟k线数据
ts.get_hist_data('600848', ktype='60') #获取60分钟k线数据
ts.get_hist_data('sh'）#获取上证指数k线数据，其它参数与个股一致，下同
ts.get_hist_data('sz'）#获取深圳成指k线数据
ts.get_hist_data('hs300'）#获取沪深300指数k线数据
ts.get_hist_data('sz50'）#获取上证50指数k线数据
ts.get_hist_data('zxb'）#获取中小板指数k线数据
ts.get_hist_data('cyb'）#获取创业板指数k线数据
```



# 复权数据

获取历史复权数据，分为前复权和后复权数据，接口提供股票上市以来所有历史数据，默认为前复权。如果不设定开始和结束日期，则返回近一年的复权数据，从性能上考虑，推荐设定开始日期和结束日期，而且最好不要超过三年以上，获取全部历史数据，请分年段分步获取，取到数据后，请及时在本地存储。获取个股首个上市日期，请参考以下方法：

df = ts.get_stock_basics()
date = df.ix['600848']['timeToMarket'] #上市日期YYYYMMDD

本接口还提供大盘指数的全部历史数据，调用时，请务必设定index参数为True,由于大盘指数不存在复权的问题，故可以忽略autype参数。
```
ts.get_h_data('002337') #前复权
ts.get_h_data('002337', autype='hfq') #后复权
ts.get_h_data('002337', autype=None) #不复权
ts.get_h_data('002337', start='2015-01-01', end='2015-03-16') #两个日期之间的前复权数据

ts.get_h_data('399106', index=True) #深圳综合指数
```

参数说明：

    code:string,股票代码 e.g. 600848
    start:string,开始日期 format：YYYY-MM-DD 为空时取当前日期
    end:string,结束日期 format：YYYY-MM-DD 为空时取去年今日
    autype:string,复权类型，qfq-前复权 hfq-后复权 None-不复权，默认为qfq
    index:Boolean，是否是大盘指数，默认为False
    retry_count : int, 默认3,如遇网络等问题重复执行的次数
    pause : int, 默认 0,重复请求数据过程中暂停的秒数，防止请求间隔时间太短出现的问题

返回值说明：

    date : 交易日期 (index)
    open : 开盘价
    high : 最高价
    close : 收盘价
    low : 最低价
    volume : 成交量
    amount : 成交金额


# 实时行情

一次性获取当前交易所有股票的行情数据（如果是节假日，即为上一交易日，结果显示速度取决于网速）

import tushare as ts

ts.get_today_all()

返回值说明：

    code：代码
    name:名称
    changepercent:涨跌幅
    trade:现价
    open:开盘价
    high:最高价
    low:最低价
    settlement:昨日收盘价
    volume:成交量
    turnoverratio:换手率
    amount:成交量
    per:市盈率
    pb:市净率
    mktcap:总市值
    nmc:流通市值


# 历史分笔

获取个股以往交易历史的分笔数据明细，通过分析分笔数据，可以大致判断资金的进出情况。在使用过程中，对于获取股票某一阶段的历史分笔数据，需要通过参入交易日参数并append到一个DataFrame或者直接append到本地同一个文件里。历史分笔接口只能获取当前交易日之前的数据，当日分笔历史数据请调用get_today_ticks()接口或者在当日18点后通过本接口获取。

参数说明：

    code：股票代码，即6位数字代码
    date：日期，格式YYYY-MM-DD
    retry_count : int, 默认3,如遇网络等问题重复执行的次数
    pause : int, 默认 0,重复请求数据过程中暂停的秒数，防止请求间隔时间太短出现的问题

调用方法：

import tushare as ts

df = ts.get_tick_data('600848',date='2014-01-09')
df.head(10)

返回值说明：

    time：时间
    price：成交价格
    change：价格变动
    volume：成交手
    amount：成交金额(元)
    type：买卖类型【买盘、卖盘、中性盘】


# 实时分笔

获取实时分笔数据，可以实时取得股票当前报价和成交信息，其中一种场景是，写一个python定时程序来调用本接口（可两三秒执行一次，性能与行情软件基本一致），然后通过DataFrame的矩阵计算实现交易监控，可实时监测交易量和价格的变化。

参数说明：

    symbols：6位数字股票代码，或者指数代码（sh=上证指数 sz=深圳成指 hs300=沪深300指数 sz50=上证50 zxb=中小板 cyb=创业板） 可输入的类型：str、list、set或者pandas的Series对象

调用方法：
```
import tushare as ts

df = ts.get_realtime_quotes('000581') #Single stock symbol
df[['code','name','price','bid','ask','volume','amount','time']]
```

结果显示：
```
   code    name     price  bid    ask    volume   amount        time
0  000581  威孚高科  31.15  31.14  31.15  8183020  253494991.16  11:30:36
```

返回值说明：
```
0：name，股票名字
1：open，今日开盘价
2：pre_close，昨日收盘价
3：price，当前价格
4：high，今日最高价
5：low，今日最低价
6：bid，竞买价，即“买一”报价
7：ask，竞卖价，即“卖一”报价
8：volume，成交量 maybe you need do volume/100
9：amount，成交金额（元 CNY）
10：b1_v，委买一（笔数 bid volume）
11：b1_p，委买一（价格 bid price）
12：b2_v，“买二”
13：b2_p，“买二”
14：b3_v，“买三”
15：b3_p，“买三”
16：b4_v，“买四”
17：b4_p，“买四”
18：b5_v，“买五”
19：b5_p，“买五”
20：a1_v，委卖一（笔数 ask volume）
21：a1_p，委卖一（价格 ask price）
...
30：date，日期；
31：time，时间；
```

请求多个股票方法（一次最好不要超过30个）：
```
#symbols from a list
ts.get_realtime_quotes(['600848','000980','000981'])
#from a Series
ts.get_realtime_quotes(df['code'].tail(10))  #一次获取10个股票的实时分笔数据
```

获取实时指数：

```
#上证指数
ts.get_realtime_quotes('sh')
#上证指数 深圳成指 沪深300指数 上证50 中小板 创业板
ts.get_realtime_quotes(['sh','sz','hs300','sz50','zxb','cyb'])
#或者混搭
ts.get_realtime_quotes(['sh','600848'])
```

# 当日历史分笔

获取当前交易日（交易进行中使用）已经产生的分笔明细数据。

参数说明：

    code：股票代码，即6位数字代码
    retry_count : int, 默认3,如遇网络等问题重复执行的次数
    pause : int, 默认 0,重复请求数据过程中暂停的秒数，防止请求间隔时间太短出现的问题

调用方法：

import tushare as ts

df = ts.get_today_ticks('601333')
df.head(10)

返回值说明：

    time：时间
    price：当前价格
    pchange:涨跌幅
    change：价格变动
    volume：成交手
    amount：成交金额(元)
    type：买卖类型【买盘、卖盘、中性盘】
